In [1]:
import gym
import numpy as np
import renom as rm
import matplotlib.pyplot as plt
from renom.utility.initializer import Gaussian
from renom.cuda import set_cuda_active
from renom_rl.dqn import DQN
from renom_rl.env import BaseEnv
from gym.core import Env
from PIL import Image

set_cuda_active(True)
env = gym.make('BreakoutNoFrameskip-v4')

class CustomEnv(BaseEnv):
    
    def __init__(self, env):
        self.env = env
        self.action_shape = 4
        self.state_shape = (4, 84, 84)
        self.previous_frames = []
        super(CustomEnv, self).__init__()
    
    def reset(self):
        self.env.reset()
        n_step = np.random.randint(4, 32+1)
        for _ in range(n_step):
            state, _, _ = self.step(self.env.action_space.sample())
        return state
    
    def sample(self):
        return self.env.action_space.sample()
    
    def render(self):
        self.env.render()

    def _preprocess(self,state):
        resized_image = Image.fromarray(state).resize((84,110)).convert('L')
        image_array = np.asarray(resized_image.getdata()).reshape(110, 84)/255.
        final_image = image_array[26:110, :]
        return final_image
    
    def step(self, action):
        state_list = []
        reward_list = []
        terminal = False
        for _ in range(4):
            # Use last frame. Other frames will be skipped.
            s, r, t, _ = self.env.step(action)
            state = self._preprocess(s)
            reward_list.append(r)
            if t:
                terminal = True
                
        if len(self.previous_frames) > 3:
            self.previous_frames = self.previous_frames[1:] + [state]
        else:
            self.previous_frames += [state]
        state = np.stack(self.previous_frames)
        return state, (np.sum(reward_list) > 0).astype(np.int), terminal
    
custom_env = CustomEnv(env)
q_network = rm.Sequential([rm.Conv2d(32, filter=8, stride=4),
                           rm.Relu(),
                           rm.Conv2d(64, filter=4, stride=2), 
                           rm.Relu(),
                           rm.Conv2d(64, filter=3, stride=1), 
                           rm.Relu(), 
                           rm.Flatten(), 
                           rm.Dense(512),
                           rm.Relu(),
                           rm.Dense(custom_env.action_shape)])


In [ ]:
model = DQN(custom_env, q_network)

In [ ]:
model.train(render=True, greedy_step=100000)

Run random 5000 step for storing experiences


episode 001 avg_loss: 0.004 total_reward [train:2.000 test:-] e-greedy:0.002: : 185it [00:05, 38.09it/s]
episode 002 avg_loss: 0.003 total_reward [train:4.000 test:-] e-greedy:0.004: : 261it [00:06, 37.39it/s]
episode 003 avg_loss: 0.004 total_reward [train:0.000 test:-] e-greedy:0.005: : 117it [00:03, 36.57it/s]
episode 004 avg_loss: 0.004 total_reward [train:3.000 test:-] e-greedy:0.007: : 226it [00:06, 37.56it/s]
episode 005 avg_loss: 0.004 total_reward [train:1.000 test:-] e-greedy:0.009: : 166it [00:04, 37.87it/s]
episode 006 avg_loss: 0.003 total_reward [train:2.000 test:-] e-greedy:0.010: : 169it [00:04, 38.05it/s]
episode 007 avg_loss: 0.003 total_reward [train:0.000 test:-] e-greedy:0.011: : 102it [00:02, 37.33it/s]
episode 008 avg_loss: 0.003 total_reward [train:2.000 test:-] e-greedy:0.013: : 185it [00:05, 36.65it/s]
episode 009 avg_loss: 0.003 total_reward [train:1.000 test:-] e-greedy:0.014: : 141it [00:03, 36.83it/s]
episode 010 avg_loss: 0.003 total_reward [train:0.000 t

episode 079 avg_loss: 0.001 total_reward [train:1.000 test:-] e-greedy:0.120: : 149it [00:04, 31.59it/s]
episode 080 avg_loss: 0.002 total_reward [train:2.000 test:-] e-greedy:0.122: : 202it [00:06, 32.00it/s]
episode 081 avg_loss: 0.002 total_reward [train:2.000 test:-] e-greedy:0.123: : 173it [00:05, 31.64it/s]
episode 082 avg_loss: 0.001 total_reward [train:2.000 test:-] e-greedy:0.125: : 181it [00:05, 31.88it/s]
episode 083 avg_loss: 0.002 total_reward [train:1.000 test:-] e-greedy:0.126: : 135it [00:04, 30.90it/s]
episode 084 avg_loss: 0.002 total_reward [train:3.000 test:-] e-greedy:0.128: : 215it [00:05, 35.90it/s]
episode 085 avg_loss: 0.002 total_reward [train:0.000 test:-] e-greedy:0.129: : 133it [00:03, 36.66it/s]
episode 086 avg_loss: 0.001 total_reward [train:2.000 test:-] e-greedy:0.131: : 201it [00:05, 37.42it/s]
episode 087 avg_loss: 0.001 total_reward [train:3.000 test:-] e-greedy:0.133: : 210it [00:05, 36.48it/s]
episode 088 avg_loss: 0.002 total_reward [train:1.000 t

episode 156 avg_loss: 0.001 total_reward [train:3.000 test:-] e-greedy:0.259: : 224it [00:06, 36.00it/s]
episode 157 avg_loss: 0.001 total_reward [train:2.000 test:-] e-greedy:0.261: : 194it [00:05, 36.93it/s]
episode 158 avg_loss: 0.001 total_reward [train:2.000 test:-] e-greedy:0.263: : 173it [00:04, 37.83it/s]
episode 159 avg_loss: 0.001 total_reward [train:4.000 test:-] e-greedy:0.265: : 247it [00:06, 37.24it/s]
episode 160 avg_loss: 0.001 total_reward [train:5.000 test:-] e-greedy:0.267: : 297it [00:07, 37.35it/s]
episode 161 avg_loss: 0.001 total_reward [train:2.000 test:-] e-greedy:0.269: : 179it [00:04, 37.86it/s]
episode 162 avg_loss: 0.003 total_reward [train:2.000 test:-] e-greedy:0.271: : 178it [00:04, 36.86it/s]
episode 163 avg_loss: 0.002 total_reward [train:3.000 test:-] e-greedy:0.273: : 213it [00:05, 37.30it/s]
episode 164 avg_loss: 0.002 total_reward [train:4.000 test:-] e-greedy:0.275: : 301it [00:08, 37.25it/s]
episode 165 avg_loss: 0.002 total_reward [train:2.000 t

episode 233 avg_loss: 0.001 total_reward [train:3.000 test:-] e-greedy:0.426: : 226it [00:06, 36.39it/s]
episode 234 avg_loss: 0.002 total_reward [train:2.000 test:-] e-greedy:0.428: : 183it [00:04, 36.87it/s]
episode 235 avg_loss: 0.002 total_reward [train:3.000 test:-] e-greedy:0.430: : 234it [00:06, 37.81it/s]
episode 236 avg_loss: 0.001 total_reward [train:2.000 test:-] e-greedy:0.432: : 201it [00:05, 36.90it/s]
episode 237 avg_loss: 0.001 total_reward [train:3.000 test:-] e-greedy:0.433: : 197it [00:05, 37.32it/s]
episode 238 avg_loss: 0.001 total_reward [train:5.000 test:-] e-greedy:0.436: : 311it [00:08, 37.34it/s]
episode 239 avg_loss: 0.001 total_reward [train:4.000 test:-] e-greedy:0.439: : 270it [00:07, 37.62it/s]
episode 240 avg_loss: 0.002 total_reward [train:5.000 test:-] e-greedy:0.441: : 299it [00:08, 36.68it/s]
episode 241 avg_loss: 0.001 total_reward [train:3.000 test:-] e-greedy:0.443: : 202it [00:05, 36.94it/s]
episode 242 avg_loss: 0.001 total_reward [train:3.000 t

...

episode 9615 avg_loss: 0.003 total_reward [train:3.000 test:-] e-greedy:0.900: : 229it [00:06, 36.20it/s]
episode 9616 avg_loss: 0.002 total_reward [train:7.000 test:-] e-greedy:0.900: : 341it [00:09, 36.61it/s]
episode 9617 avg_loss: 0.003 total_reward [train:6.000 test:-] e-greedy:0.900: : 324it [00:08, 37.56it/s]
episode 9618 avg_loss: 0.003 total_reward [train:4.000 test:-] e-greedy:0.900: : 267it [00:07, 36.50it/s]
episode 9619 avg_loss: 0.003 total_reward [train:5.000 test:-] e-greedy:0.900: : 272it [00:07, 36.79it/s]
episode 9620 avg_loss: 0.003 total_reward [train:6.000 test:-] e-greedy:0.900: : 308it [00:08, 36.94it/s]
episode 9621 avg_loss: 0.003 total_reward [train:5.000 test:-] e-greedy:0.900: : 298it [00:08, 36.57it/s]
episode 9622 avg_loss: 0.003 total_reward [train:4.000 test:-] e-greedy:0.900: : 256it [00:07, 36.89it/s]
episode 9623 avg_loss: 0.003 total_reward [train:5.000 test:-] e-greedy:0.900: : 291it [00:08, 36.35it/s]
episode 9624 avg_loss: 0.003 total_reward [tra

episode 9692 avg_loss: 0.003 total_reward [train:6.000 test:-] e-greedy:0.900: : 372it [00:10, 37.08it/s]
episode 9693 avg_loss: 0.003 total_reward [train:6.000 test:-] e-greedy:0.900: : 312it [00:08, 36.48it/s]
episode 9694 avg_loss: 0.003 total_reward [train:6.000 test:-] e-greedy:0.900: : 339it [00:09, 34.97it/s]
episode 9695 avg_loss: 0.003 total_reward [train:5.000 test:-] e-greedy:0.900: : 322it [00:09, 34.65it/s]
episode 9696 avg_loss: 0.003 total_reward [train:8.000 test:-] e-greedy:0.900: : 411it [00:11, 36.00it/s]
episode 9697 avg_loss: 0.003 total_reward [train:4.000 test:-] e-greedy:0.900: : 267it [00:07, 35.91it/s]
episode 9698 avg_loss: 0.003 total_reward [train:8.000 test:-] e-greedy:0.900: : 417it [00:11, 36.11it/s]
episode 9699 avg_loss: 0.003 total_reward [train:8.000 test:-] e-greedy:0.900: : 464it [00:12, 36.65it/s]
episode 9700 avg_loss: 0.003 total_reward [train:6.000 test:4.000] e-greedy:0.900: : 368it [00:12, 35.78it/s]
episode 9701 avg_loss: 0.003 total_reward 

episode 9769 avg_loss: 0.003 total_reward [train:7.000 test:-] e-greedy:0.900: : 359it [00:09, 36.55it/s]
episode 9770 avg_loss: 0.003 total_reward [train:6.000 test:-] e-greedy:0.900: : 310it [00:08, 36.27it/s]
episode 9771 avg_loss: 0.003 total_reward [train:6.000 test:-] e-greedy:0.900: : 313it [00:08, 36.24it/s]
episode 9772 avg_loss: 0.003 total_reward [train:7.000 test:-] e-greedy:0.900: : 411it [00:11, 36.28it/s]
episode 9773 avg_loss: 0.003 total_reward [train:10.000 test:-] e-greedy:0.900: : 502it [00:13, 36.64it/s]
episode 9774 avg_loss: 0.003 total_reward [train:10.000 test:-] e-greedy:0.900: : 785it [00:21, 36.77it/s]
episode 9775 avg_loss: 0.003 total_reward [train:4.000 test:-] e-greedy:0.900: : 254it [00:06, 36.53it/s]
episode 9776 avg_loss: 0.003 total_reward [train:4.000 test:-] e-greedy:0.900: : 244it [00:06, 36.65it/s]
episode 9777 avg_loss: 0.003 total_reward [train:10.000 test:-] e-greedy:0.900: : 507it [00:13, 36.87it/s]
episode 9778 avg_loss: 0.003 total_reward [

episode 9846 avg_loss: 0.003 total_reward [train:3.000 test:-] e-greedy:0.900: : 206it [00:05, 35.41it/s]
episode 9847 avg_loss: 0.003 total_reward [train:4.000 test:-] e-greedy:0.900: : 253it [00:07, 35.10it/s]
episode 9848 avg_loss: 0.003 total_reward [train:8.000 test:-] e-greedy:0.900: : 390it [00:11, 35.36it/s]
episode 9849 avg_loss: 0.003 total_reward [train:8.000 test:-] e-greedy:0.900: : 412it [00:11, 36.32it/s]
episode 9850 avg_loss: 0.003 total_reward [train:6.000 test:4.000] e-greedy:0.900: : 314it [00:10, 29.61it/s]
episode 9851 avg_loss: 0.003 total_reward [train:9.000 test:-] e-greedy:0.900: : 460it [00:13, 35.53it/s]
episode 9852 avg_loss: 0.003 total_reward [train:4.000 test:-] e-greedy:0.900: : 252it [00:07, 35.44it/s]
episode 9853 avg_loss: 0.003 total_reward [train:8.000 test:-] e-greedy:0.900: : 408it [00:11, 34.83it/s]
episode 9854 avg_loss: 0.003 total_reward [train:7.000 test:-] e-greedy:0.900: : 415it [00:11, 35.11it/s]
episode 9855 avg_loss: 0.003 total_reward 

episode 9923 avg_loss: 0.003 total_reward [train:4.000 test:-] e-greedy:0.900: : 242it [00:06, 36.77it/s]
episode 9924 avg_loss: 0.003 total_reward [train:10.000 test:-] e-greedy:0.900: : 470it [00:12, 36.21it/s]
episode 9925 avg_loss: 0.003 total_reward [train:4.000 test:-] e-greedy:0.900: : 257it [00:07, 34.86it/s]
episode 9926 avg_loss: 0.003 total_reward [train:7.000 test:-] e-greedy:0.900: : 377it [00:10, 36.52it/s]
episode 9927 avg_loss: 0.003 total_reward [train:6.000 test:-] e-greedy:0.900: : 314it [00:08, 36.14it/s]
episode 9928 avg_loss: 0.003 total_reward [train:11.000 test:-] e-greedy:0.900: : 737it [00:20, 36.70it/s]
episode 9929 avg_loss: 0.003 total_reward [train:6.000 test:-] e-greedy:0.900: : 338it [00:09, 36.62it/s]
episode 9930 each step reward:2.000: : 105it [00:02, 36.52it/s]

In [ ]:
import time
start_t = time.time()
a = np.random.permutation(int(1e1))
print(time.time()-start_t)

# 